In [1]:
!pip install swifter --user
!pip install numpy --user
!pip install nltk --user
!pip install sklearn --user
!pip install arrow --user

In [1]:
import json 
import numpy as np
import pandas as pd
import re
import string
import nltk
import csv
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer 
import gc
import swifter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle 
import sys
import arrow
import numpy as np
from numpy.linalg import norm
from multiprocessing import Pool
import correct
from correct import correct_numbers

In [2]:
# load nltk data
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zeelt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zeelt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
STOP_WORDS = stopwords.words('english') # stop words
lemmatizer = WordNetLemmatizer() # lemmatizer
tokenizer = RegexpTokenizer(r'\w+') # tokenizer (lexer)

In [8]:
# importing dataset
df_business = pd.read_csv('yelp_business.csv')
df_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [9]:
df_business = df_business[df_business['categories'].notna() & df_business['state'].notna()]
df_business_restaurants = df_business[df_business['categories'].str.contains('Restaurants') & df_business['state'].str.contains('NC|SC')] 

# delete df_business and call the garbage collector
del(df_business)
gc.collect()

# leaving only open establishments (1 = open , 0 = closed)
df_business_restaurants = df_business_restaurants[df_business_restaurants['is_open'] == 1]
df_business_restaurants.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
103,c7X2SdKxVJMaOnFROO8WEg,"""Finga Lickin' Caribbean Eatery""",NaN,"""2838 The Plz""",Charlotte,NC,28205,35.236823,-80.801084,4.5,21,1,Pizza;Food;Internet Cafes;Restaurants;Caribbean
126,WUiDaFQRZ8wKYGLvmjFjAw,"""China Buffet""",University City,"""8630 University Executive Park Dr""",Charlotte,NC,28262,35.306173,-80.752672,3.5,76,1,Buffets;Restaurants;Sushi Bars;Chinese
128,5q6Xh-UcJa78bp6dzyaE7w,"""Duck Donuts""",Dilworth,"""1710 Kenilworth Ave, Ste 220""",Charlotte,NC,28203,35.202624,-80.844419,4.5,373,1,Breakfast & Brunch;Food;Coffee & Tea;Donuts;Re...
161,bA21m-qbgN_GNR6g-AlfYw,"""Dynasty Buffett""",NaN,"""2932 E Franklin Blvd""",Gastonia,NC,28056,35.262735,-81.125895,3.5,30,1,Restaurants;Chinese


In [10]:
df_business_restaurants.to_csv('df_business_restaurants.csv')

In [11]:
print(df_business_restaurants.shape)

(3064, 13)


In [12]:
# word processing
def text_preprocess(text):
    text = tokenizer.tokenize(text.lower()) # we convert the text to lowercase and split it into tokens by spaces and punctuation marks
    text = [re.sub('[^a-z\s]', '', w) for w in text]# get rid of numbers and non-Latin characters
    text = [lemmatizer.lemmatize(w) for w in text if w not in STOP_WORDS]# perform lemmatization and get rid of stop words
    return ' '.join(text)

In [13]:
# feedback data
df_review = pd.read_csv("yelp_review.csv")
df_review = df_review.drop(['useful','funny','cool','date'],axis=1)
# dataframe size
print(df_review.shape)
df_review.head()

(5261668, 5)


,review_id,user_id,business_id,stars,text
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,Super simple place but amazing nonetheless. It...
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,Small unassuming place that changes their menu...
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,Lester's is located in a beautiful neighborhoo...
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,Love coming here. Yes the place always needs t...
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,Had their chocolate almond croissant and it wa...


In [14]:
# we leave only those reviews that relate to food establishments from the df_business_restaurants dataframe
df_review = df_review[df_review['business_id'].isin(df_business_restaurants['business_id'])]
# replace missing reviews with an empty string 
df_review["text"].fillna('', inplace=True)

In [15]:
# effectively process the text using swifter and clear the received dataframe from reviews with empty text
df_review['text'] = df_review['text'].swifter.apply(text_preprocess)
df_review.shape # final dataframe size

Pandas Apply:   0%|          | 0/183106 [00:00<?, ?it/s]

(183106, 5)

In [16]:
# write the resulting file without indexes
df_review.to_csv('review_data_state.csv', index=False)

In [17]:
class MatrixFactorization(object):
    
    def __init__(self, U, I, learning_rate=0.1, lamda=0.1, num_epoches=5, num_batches=100):
        self.learning_rate = learning_rate # learning rate
        self.lamda = lamda # lambda for L2 regularization 
        self.num_epoches = num_epoches  # number of epochs
        self.num_batches = num_batches  # number of batches
        self.U = U # users feature matrix 
        self.I = I # items feature matrix
        self.e = 0 # current epoch

    # loss function
    def loss(self, ratings):
        errors = [(rat_ij - np.dot(self.U[i], self.I[j].T))**2 + self.lamda * \
              (norm(self.U[i]) + norm(self.I[j])) for i, j, rat_ij in ratings]
        return sum(errors)

    # updating U and I matrices with stochastic gradient descent
    def update(self, ratings):
        for i, j, rat_ij in ratings:
            rat_ij_hat = np.dot(self.U[i], self.I[j].T)
            grad_U_i = (rat_ij_hat - rat_ij) * self.I[j] + self.lamda * self.U[i]
            grad_I_j = (rat_ij_hat - rat_ij) * self.U[i] + self.lamda * self.I[j]
            self.U[i] -= self.learning_rate * grad_U_i
            self.I[j] -= self.learning_rate * grad_I_j
    
    # model setup
    def fit(self, train, test):
        num_trains = train.shape[0] # number of training data
        num_tests  = test.shape[0] # number of test data
        while self.e < self.num_epoches:
            self.e += 1
            # shuffle training data
            mixed_order = np.arange(num_trains)
            np.random.shuffle(mixed_order)
            avg_train_loss = [] # training data error 
            avg_test_loss  = [] # test data error
            batch_size = int(num_trains / self.num_batches) # batch size
            for batch in range(self.num_batches):
                batch_indexes = np.arange(batch_size * batch, batch_size * (batch + 1))             
                # select the current batch from the training data
                current_batch_ratings = train[mixed_order[batch_indexes], :]
                # sampling of random values from the test set with a size equal to the size of the training data batch
                sample_test = test[np.random.choice(num_tests, batch_size), :]
                # updating matrices U and I
                self.update(current_batch_ratings)
                # loss of training and test data
                train_loss = self.loss(current_batch_ratings)
                test_loss  = self.loss(sample_test)
                avg_train_loss.append(train_loss)
                avg_test_loss.append(test_loss)
            
            avg_train_loss = np.mean(avg_train_loss) / float(batch_size) # average error of training data
            avg_test_loss  = np.mean(avg_test_loss) / float(batch_size) # mean error of test data
            print(f'{arrow.now()} Epoch № {self.e}', file=sys.stderr)
            print(f'{arrow.now()} Training loss:\t{avg_train_loss}', file=sys.stderr)
            print(f'{arrow.now()} Testing loss:\t{avg_test_loss}', file=sys.stderr)

In [18]:
# read csv file with processed data
df_review = pd.read_csv("review_data_state.csv")
# replace missing reviews with an empty string 
df_review["text"].fillna('', inplace=True)
print(df_review.head())
df_review = df_review[['user_id', 'business_id', 'stars', 'text']]
# matching unique user ids to numeric values
dict_users = {key: val for val, key in enumerate(df_review['user_id'].unique())}
inv_dict_users = {val: key for key, val in dict_users.items()}
# correspondence of unique id of establishments to numerical values
dict_items = {key: val for val, key in enumerate(df_review['business_id'].unique())}
inv_dict_items = {val: key for key, val in dict_items.items()}
    
df_review['user_id'] = df_review['user_id'].swifter.apply(lambda x: dict_users[x])
df_review['business_id'] = df_review['business_id'].swifter.apply(lambda x: dict_items[x])
    
# separating the data into training and test
train_ratings_df, test_ratings_df = train_test_split(df_review, test_size = 0.2)

print(f'Size of training data: {train_ratings_df.shape}')
print(f'Size of test data: {test_ratings_df.shape}')

                review_id                 user_id             business_id  \
0  WUz0DjjrEyWpHbMQOJe7gQ  3wB-JjsUjkvdNUDRnChsVg  yymNrvk1cioG5Bn5lWH2QQ   
1  xbTDtqbo8MPvc2JazgoW9g  ewoVtCaCosO1dLqlPfuzNQ  sPwgafejYVdf8hd4-oDYVg   
2  Gy55Wq_v5farLU8TyvVDlQ  ewoVtCaCosO1dLqlPfuzNQ  R1jJQi2yR44D_2ileqr8kA   
3  LdmQZwe7h3tNycKy1cR_bg  ewoVtCaCosO1dLqlPfuzNQ  RcB7STNxoSDjtoK1pXgWBg   
4  7fhTZf0zpRq9jsWneQ7IXA  ewoVtCaCosO1dLqlPfuzNQ  AuIiqDeL65kczmiFSkAWrA   

   stars                                               text  
0      2  busy night told u  minute wait ok problem went...  
1      4  wing good great price love honey hot better wi...  
2      5  came day life cant remember name ordered highl...  
3      4  im pretty impressed food fab many flavor mood ...  
4      4         best chain italian food spot charlotte far  


Pandas Apply:   0%|          | 0/183106 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/183106 [00:00<?, ?it/s]

Size of training data: (146484, 4)
Size of test data: (36622, 4)


In [19]:
# for each user and institution, we combine all reviews into one paragraph
def get_train_whole_text(df, name_id):
    return df[[name_id,'text']].groupby(name_id).agg({'text': ' '.join})
    
    
user_id_review_df = pd.DataFrame(index=inv_dict_users.keys(), columns=["text"])
user_id_review_df.index.name = 'user_id'
business_id_review_df= pd.DataFrame(index=inv_dict_items.keys(), columns=["text"])
business_id_review_df.index.name = 'business_id'

# tables store all users and establishments,
# but only those recommendation texts that are relevant to the training data
user_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'user_id')["text"]
business_id_review_df['text'] =  get_train_whole_text(train_ratings_df, 'business_id')["text"]
user_id_review_df["text"].fillna('', inplace=True)
business_id_review_df["text"].fillna('', inplace=True)

In [20]:
# extract features (feature extraction) from reviews using vectorization (TFIDF Vectorizer),
# to use text to factorize a matrix
business_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
business_id_vectors = business_id_vectorizer.fit_transform(business_id_review_df['text'])

user_id_vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize, max_features=5000)
user_id_vectors = user_id_vectorizer.fit_transform(user_id_review_df['text'])

# users-features matrix
U = pd.DataFrame(user_id_vectors.toarray(), index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
# business-features matrix
I = pd.DataFrame(business_id_vectors.toarray(), index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

matrixF = MatrixFactorization(U = U.to_numpy(), I = I.to_numpy(), learning_rate=0.07, lamda=0.1, num_epoches=8, num_batches=1000)

In [21]:
R = pd.pivot_table(df_review, values='stars', index=['user_id'], columns=['business_id'])
R.head()

business_id,0,1,2,3,4,5,6,7,8,9,...,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063
user_id,,,,,,,,,,,,,,,,,,,,,
0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,4.0,5.0,4.0,4.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
matrixF.fit(train_ratings_df.to_numpy()[:, 0:3], test_ratings_df.to_numpy()[:, 0:3])

2022-04-09T15:37:04.252557+05:30 Epoch № 1
2022-04-09T15:37:04.252557+05:30 Training loss:	1.9340961254977231
2022-04-09T15:37:04.252557+05:30 Testing loss:	7.736447450821662
2022-04-09T15:37:18.770921+05:30 Epoch № 2
2022-04-09T15:37:18.770921+05:30 Training loss:	0.5912324921740809
2022-04-09T15:37:18.770921+05:30 Testing loss:	5.775119943535401
2022-04-09T15:37:33.105427+05:30 Epoch № 3
2022-04-09T15:37:33.105427+05:30 Training loss:	0.5133931129322032
2022-04-09T15:37:33.105427+05:30 Testing loss:	5.600576459860597
2022-04-09T15:37:47.081696+05:30 Epoch № 4
2022-04-09T15:37:47.081696+05:30 Training loss:	0.4956006462557998
2022-04-09T15:37:47.081696+05:30 Testing loss:	5.544917078507142
2022-04-09T15:38:01.480089+05:30 Epoch № 5
2022-04-09T15:38:01.480089+05:30 Training loss:	0.48852420801572755
2022-04-09T15:38:01.480089+05:30 Testing loss:	5.546312753743678
2022-04-09T15:38:15.647282+05:30 Epoch № 6
2022-04-09T15:38:15.647282+05:30 Training loss:	0.4841730638193551
2022-04-09T15:

In [23]:
U = pd.DataFrame(matrixF.U, index=user_id_review_df.index, columns=user_id_vectorizer.get_feature_names())
U.sort_index(inplace=True)
I = pd.DataFrame(matrixF.I, index=business_id_review_df.index, columns=business_id_vectorizer.get_feature_names())
I.sort_index(inplace=True)

In [24]:
if __name__ == '__main__':
    R = np.dot(matrixF.U, matrixF.I.T)
    with Pool(processes = 10) as pool:
        R = np.array(pool.map(correct_numbers, R))

In [25]:
# predicted ratings
R = pd.DataFrame(R, index=U.index, columns=I.index)

In [26]:
print(f"Predicted ratings:\n")
R.head()

Predicted ratings:



business_id,0,1,2,3,4,5,6,7,8,9,...,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063
user_id,,,,,,,,,,,,,,,,,,,,,
0,2,2,3,3,3,3,3,3,3,3,...,1,2,2,1,2,3,1,1,1,1
1,3,4,5,4,4,5,4,4,4,4,...,2,3,2,2,2,4,1,1,2,2
2,2,4,4,4,4,4,5,3,5,5,...,2,3,2,2,2,3,1,1,2,2
3,2,3,4,3,3,3,3,3,3,4,...,1,2,2,1,2,3,1,1,1,1
4,2,3,3,3,3,3,4,3,3,3,...,1,2,2,1,2,3,1,1,1,1


In [27]:
# top 10 signs for the first institution
I.iloc[train_ratings_df.iat[0, 1]].sort_values(ascending=False).head(10)

place      0.538708
good       0.499796
pastry     0.462414
love       0.448617
food       0.435405
brownie    0.430245
great      0.417556
caramel    0.406781
salted     0.404390
amelie     0.392790
Name: 382, dtype: float64

In [28]:
# top 10 signs for the first user
U.iloc[train_ratings_df.iat[0, 0]].sort_values(ascending=False).head(10)

good       0.396725
food       0.334373
place      0.333249
amelie     0.304033
time       0.293293
rooster    0.286795
great      0.268817
love       0.263502
go         0.262274
really     0.257570
Name: 28480, dtype: float64

In [29]:
# save matrices U, I and vectorizer
data = {'U': U, 'I': I, 'V': user_id_vectorizer}

with open('recommendation_model.pkl', 'wb') as f:
     pickle.dump(data, f)

In [30]:
# write the resulting file without indexes
R.to_csv('predicted_ratings.csv', sep='\t')

In [31]:
# loading the saved model
with open('recommendation_model.pkl', 'rb') as f:
     model = pickle.load(f)
print(model)

{'U':           ability      able  absolute  absolutely  abundance  abundant  \
user_id                                                                  
0       -0.000096  0.014123  0.003683    0.037908   0.005368  0.001741   
1        0.002981  0.012956  0.013881    0.068765   0.001744  0.001467   
2        0.002659  0.036590  0.003817    0.026507  -0.000578  0.001176   
3        0.000545  0.040091 -0.007044    0.056253   0.000749  0.000370   
4       -0.002124  0.013286  0.016403    0.057184   0.000086  0.001189   
...           ...       ...       ...         ...        ...       ...   
61043   -0.002095  0.004766  0.004503    0.005273   0.000745  0.000459   
61044    0.000114  0.006468  0.001033    0.008518   0.000857  0.000203   
61045    0.000574  0.038484 -0.002430    0.099503   0.000433  0.005434   
61046    0.001993  0.014268  0.003755    0.028980   0.003297  0.000539   
61047    0.012789  0.014528  0.002598    0.029475   0.000987 -0.001217   

               ac        accent

In [32]:
I = model["I"]
U = model["U"]
user_id_vectorizer = model["V"] 

In [33]:
R = pd.read_csv('predicted_ratings.csv', sep='\t')
R.head()

,user_id,0,1,2,3,4,5,6,7,8,...,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063
0,0,2,2,3,3,3,3,3,3,3,...,1,2,2,1,2,3,1,1,1,1
1,1,3,4,5,4,4,5,4,4,4,...,2,3,2,2,2,4,1,1,2,2
2,2,2,4,4,4,4,4,5,3,5,...,2,3,2,2,2,3,1,1,2,2
3,3,2,3,4,3,3,3,3,3,3,...,1,2,2,1,2,3,1,1,1,1
4,4,2,3,3,3,3,3,4,3,3,...,1,2,2,1,2,3,1,1,1,1


In [34]:
words = input("What place are you looking for? ")
#words = "Italian dinner with beautiful views and tasty wine"


query = pd.DataFrame([text_preprocess(words)], columns=['text']) # create a dataframe with a given query
query_vector = user_id_vectorizer.transform(query['text']) # selection of query features
query = pd.DataFrame(query_vector.toarray(), index=query.index, columns=user_id_vectorizer.get_feature_names())

predictRating=pd.DataFrame(np.dot(query.loc[0],I.T), index=I.index, columns=['rating'])
topRecommendations=pd.DataFrame.sort_values(predictRating,['rating'],ascending=[0])[:5]

number = 0
for i in topRecommendations.index:
    number+=1
    print(f"\n№ {number}")
    print(f"Name: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['name'].iloc[0]}")
    print(f"City, state: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['city'].iloc[0]}, {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['state'].iloc[0]}")
    print(f"Category: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['categories'].iloc[0]}")
    print(f"Average star: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['stars'].iloc[0]}")
    print(f"Review count: {df_business_restaurants[df_business_restaurants['business_id']==inv_dict_items[i]]['review_count'].iloc[0]}")

What place are you looking for? Italian dinner with beautiful views and tasty wine

№ 1
Name: "Fahrenheit"
City, state: Charlotte, NC
Category: American (New);Pizza;Salad;Restaurants
Average star: 3.0
Review count: 523

№ 2
Name: "Dilworth Tasting Room"
City, state: Charlotte, NC
Category: Nightlife;Wineries;Restaurants;Bars;Wine Tasting Room;Food;Arts & Entertainment;Tapas/Small Plates;Wine Bars;Beer;Wine & Spirits;Modern European
Average star: 4.5
Review count: 77

№ 3
Name: "Antico Italian Restaurant"
City, state: Huntersville, NC
Category: Italian;Restaurants;Pizza;Seafood
Average star: 4.5
Review count: 83

№ 4
Name: "Lake Norman Cottage"
City, state: Davidson, NC
Category: Food;Beer;Wine & Spirits;Nightlife;Restaurants;Wine Bars;Bars;Tapas/Small Plates
Average star: 4.0
Review count: 25

№ 5
Name: "Ferrucci's Old Tyme Italian Market"
City, state: Cornelius, NC
Category: Italian;Delis;Meat Shops;Food;Restaurants;Caterers;Event Planning & Services;Grocery;Specialty Food
Average sta